In [5]:
import ensembl_rest

In [4]:
x = ensembl_rest.sequence_region(species='human', region='19:7924986..7926086:-1')['seq']

In [5]:
'AGCGCGACGTGGACG' in x

True

In [9]:
#ensembl_rest.lookup('ENSG00000178531', params={'expand': True})

In [12]:
#ensembl_rest.xref_id('ENSG00000127922')

In [15]:
x = ensembl_rest.lookup('ENSG00000127922', params={'expand': True})

In [4]:
import pandas
import numpy

In [64]:
missings = pandas.read_excel('missing_nextprot_to_ensembl.xlsx')

In [65]:
missings[missings['From'] == 'Q6ZVN7']

,From,To
859,Q6ZVN7,ENST00000356686


In [20]:
hek = pandas.read_csv('transcript_rna_celline.tsv', sep='\t')

In [21]:
hek['HEK293'] = numpy.mean([hek['TPM.HEK 293.9'].to_numpy(), hek['TPM.HEK 293.10'].to_numpy()], axis=0)

In [22]:
hek = hek[['ensgid', 'enstid', 'HEK293']]

In [69]:
hek = hek[hek['HEK293'] != 0.0]

In [70]:
missings = missings[missings['To'].isin(hek['enstid'])]

In [59]:
'Q6ZVN7' in missings['From'].to_list()

False

In [23]:
missings['HEK293_TPM'] = missings['ENST'].map(hek.set_index('enstid')['HEK293'])

In [74]:
missings.sort_values(by='HEK293_TPM', ascending=False)

,From,To,HEK293_TPM
491,P0DMP1,ENST00000600213,3188.445000
470,P0CJ75,ENST00000536684,547.973500
463,P0CJ68,ENST00000540040,393.955500
187,A4D0T7,ENST00000397764,93.494150
591,P60606,ENST00000318978,60.018250
...,...,...,...
1379,Q9GZK7,ENST00000549749,0.003629
482,P0DJD0,ENST00000409776,0.000296
968,Q8N628,ENST00000641802,0.000122
408,O76099,ENST00000642030,0.000082


In [1]:
import requests

In [2]:
with open('missings_entries.txt') as f:
    entries = f.read().splitlines()

In [10]:
comb2 = []
for i in range((len(entries)//100)+1):
    query = ' '.join(entries[i*100:min((i+1)*100, len(entries))])
    r = requests.get('https://www.uniprot.org/uploadlists', params={'from': 'ACC+ID',
                                                                'to': 'ENSEMBL_TRS_ID',
                                                                'format': 'tab',
                                                                'taxon': '9606',
                                                                'query': query})
    comb2.append(r.text)

In [11]:
#comb2

In [12]:
comb2 = '\n'.join(comb2)

In [15]:
comb = comb2.replace('From\tTo\n', '')

In [16]:
from io import StringIO

In [17]:
missings = pandas.read_csv(StringIO(comb), sep='\t', header=None)

In [18]:
missings.columns=['AC', 'ENST']

In [25]:
missings = missings[missings['HEK293_TPM'] != 0.0]

In [31]:
entries2 = list(missings['AC'].unique())

In [70]:
#entries2

In [33]:
comb = []
for i in range((len(entries2)//100)+1):
    query = ' '.join(entries2[i*100:min((i+1)*100, len(entries2))])
    r = requests.get('https://www.uniprot.org/uploadlists', params={'from': 'ACC+ID',
                                                                'to': 'ACC',
                                                                'format': 'tab',
                                                                'taxon': '9606',
                                                                'columns': 'id,entry_name,reviewed,genes,length',
                                                                'query': query})
    comb.append(r.text)

In [69]:
#comb

In [35]:
comb = [i.split('\n')[1:-1] for i in comb]

In [36]:
comb = [item for sublist in comb for item in sublist]

In [37]:
len(comb)

328

In [39]:
comb = '\n'.join(comb)

In [40]:
combs = pandas.read_csv(StringIO(comb), sep='\t', header=None)

In [41]:
missings['Entry'] = missings['AC'].map(combs.set_index('0')['1'])

,0,1,2,3,4,5
0,A0A075B6X5,TVA18_HUMAN,reviewed,TRAV18,111,A0A075B6X5
1,A0A087WV53,SPEGN_HUMAN,reviewed,SPEGNB,238,A0A087WV53
2,A0A096LNP1,D131B_HUMAN,reviewed,DEFB131B,70,A0A096LNP1
3,A0A096LP55,QCR6L_HUMAN,reviewed,UQCRHL,91,A0A096LP55
4,A0A0D9SF12,CC163_HUMAN,reviewed,CCDC163 C1orf231 CCDC163P,145,A0A0D9SF12
...,...,...,...,...,...,...
323,Q9Y543,HES2_HUMAN,reviewed,HES2 BHLHB40,173,Q9Y543
324,Q9Y6U7,RN215_HUMAN,reviewed,RNF215,377,Q9Y6U7
325,S4R3P1,HMN13_HUMAN,reviewed,MTRNR2L13,24,S4R3P1
326,S4R3Y5,HMN11_HUMAN,reviewed,MTRNR2L11,24,S4R3Y5


In [43]:
missings['Entry'] = missings['AC'].map(combs.set_index(0)[1])

C:\Users\Dynev\AppData\Local\Temp\ipykernel_15056\2318448750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missings['Entry'] = missings['AC'].map(combs.set_index(0)[1])


In [45]:
missings['Reviewed'] = missings['AC'].map(combs.set_index(0)[2])

C:\Users\Dynev\AppData\Local\Temp\ipykernel_15056\773351778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missings['Reviewed'] = missings['AC'].map(combs.set_index(0)[2])


In [46]:
missings['Gene'] = missings['AC'].map(combs.set_index(0)[3])

C:\Users\Dynev\AppData\Local\Temp\ipykernel_15056\347189463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missings['Gene'] = missings['AC'].map(combs.set_index(0)[3])


In [47]:
missings['Length'] = missings['AC'].map(combs.set_index(0)[4])

C:\Users\Dynev\AppData\Local\Temp\ipykernel_15056\4230732883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missings['Length'] = missings['AC'].map(combs.set_index(0)[4])


In [48]:
missings

,AC,ENST,HEK293_TPM,Entry,Reviewed,Gene,Length
6,A0A075B6X5,ENST00000390446,0.284602,TVA18_HUMAN,reviewed,TRAV18,111
14,A0A087WV53,ENST00000617028,0.309398,SPEGN_HUMAN,reviewed,SPEGNB,238
25,A0A096LNP1,ENST00000530210,2.549905,D131B_HUMAN,reviewed,DEFB131B,70
26,A0A096LP55,ENST00000483273,0.096707,QCR6L_HUMAN,reviewed,UQCRHL,91
67,A0A0D9SF12,ENST00000629482,1.484775,CC163_HUMAN,reviewed,CCDC163 C1orf231 CCDC163P,145
...,...,...,...,...,...,...,...
1532,Q9Y543,ENST00000377834,0.069505,HES2_HUMAN,reviewed,HES2 BHLHB40,173
1537,Q9Y6U7,ENST00000382363,4.730035,RN215_HUMAN,reviewed,RNF215,377
1538,S4R3P1,ENST00000604093,0.096318,HMN13_HUMAN,reviewed,MTRNR2L13,24
1539,S4R3Y5,ENST00000604646,36.966950,HMN11_HUMAN,reviewed,MTRNR2L11,24


In [57]:
missings = missings[['AC', 'Entry', 'Gene', 'ENST', 'HEK293_TPM']]

In [68]:
missings.to_excel('missings_enst_hek.xlsx', index=False)

In [66]:
missings = missings.dropna()

In [62]:
'Q8N812' in missings['AC'].to_list()

False

In [64]:
hek[hek['enstid'] == 'ENST00000683810']

,ensgid,enstid,HEK293


In [98]:
x = ensembl_rest.xref_id('ENSG00000286095', all_levels=True)

In [110]:
ensembl_rest.xref_external(species = 'human', symbol='P60606')

[{'type': 'gene', 'id': 'ENSG00000178531'},
 {'id': 'ENST00000318978', 'type': 'transcript'},
 {'id': 'ENSP00000313226', 'type': 'translation'}]

In [74]:
ensps = []
for i in missings['ENST'].to_list():
    try:
        ensps.append(ensembl_rest.lookup(i, params={'expand': True})['Translation']['id'])
    except:
        ensps.append('')

In [76]:
missings['ENSP'] = ensps

C:\Users\Dynev\AppData\Local\Temp\ipykernel_15056\169233238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missings['ENSP'] = ensps


In [77]:
missings

,AC,Entry,Gene,ENST,HEK293_TPM,ENSP
6,A0A075B6X5,TVA18_HUMAN,TRAV18,ENST00000390446,0.284602,ENSP00000451574
14,A0A087WV53,SPEGN_HUMAN,SPEGNB,ENST00000617028,0.309398,
25,A0A096LNP1,D131B_HUMAN,DEFB131B,ENST00000530210,2.549905,ENSP00000485141
26,A0A096LP55,QCR6L_HUMAN,UQCRHL,ENST00000483273,0.096707,ENSP00000485401
67,A0A0D9SF12,CC163_HUMAN,CCDC163 C1orf231 CCDC163P,ENST00000629482,1.484775,ENSP00000486197
...,...,...,...,...,...,...
1532,Q9Y543,HES2_HUMAN,HES2 BHLHB40,ENST00000377834,0.069505,ENSP00000367065
1537,Q9Y6U7,RN215_HUMAN,RNF215,ENST00000382363,4.730035,ENSP00000371800
1538,S4R3P1,HMN13_HUMAN,MTRNR2L13,ENST00000604093,0.096318,ENSP00000474570
1539,S4R3Y5,HMN11_HUMAN,MTRNR2L11,ENST00000604646,36.966950,ENSP00000474861


In [108]:
missings

,AC,Entry,Gene,ENST,HEK293_TPM,ENSP
6,A0A075B6X5,TVA18_HUMAN,TRAV18,ENST00000390446,0.284602,ENSP00000451574
14,A0A087WV53,SPEGN_HUMAN,SPEGNB,ENST00000617028,0.309398,
25,A0A096LNP1,D131B_HUMAN,DEFB131B,ENST00000530210,2.549905,ENSP00000485141
26,A0A096LP55,QCR6L_HUMAN,UQCRHL,ENST00000483273,0.096707,ENSP00000485401
67,A0A0D9SF12,CC163_HUMAN,CCDC163 C1orf231 CCDC163P,ENST00000629482,1.484775,ENSP00000486197
...,...,...,...,...,...,...
1532,Q9Y543,HES2_HUMAN,HES2 BHLHB40,ENST00000377834,0.069505,ENSP00000367065
1537,Q9Y6U7,RN215_HUMAN,RNF215,ENST00000382363,4.730035,ENSP00000371800
1538,S4R3P1,HMN13_HUMAN,MTRNR2L13,ENST00000604093,0.096318,ENSP00000474570
1539,S4R3Y5,HMN11_HUMAN,MTRNR2L11,ENST00000604646,36.966950,ENSP00000474861


In [116]:
def ensembl_ac_to_trs(ac):
    x = ensembl_rest.xref_external(species='human', symbol=ac)
    x = [i['id'] for i in x if i['type'] == 'transcript']
    return x

def ensembl

In [117]:
ensembl_ac_to_trs('Q6ZVN7')

['ENST00000356686']

In [124]:
ensembl_rest.lookup('ENST00000390446', params={'expand':True})

{'end': 22003673,
 'version': 3,
 'strand': 1,
 'logic_name': 'havana_ig_gene_homo_sapiens',
 'id': 'ENST00000390446',
 'db_type': 'core',
 'Exon': [{'assembly_name': 'GRCh38',
   'start': 22003106,
   'seq_region_name': '14',
   'end': 22003256,
   'version': 2,
   'object_type': 'Exon',
   'strand': 1,
   'species': 'homo_sapiens',
   'id': 'ENSE00001851593',
   'db_type': 'core'},
  {'strand': 1,
   'object_type': 'Exon',
   'end': 22003673,
   'version': 3,
   'db_type': 'core',
   'id': 'ENSE00001944147',
   'species': 'homo_sapiens',
   'assembly_name': 'GRCh38',
   'seq_region_name': '14',
   'start': 22003385}],
 'Translation': {'Parent': 'ENST00000390446',
  'version': 1,
  'object_type': 'Translation',
  'end': 22003673,
  'id': 'ENSP00000451574',
  'db_type': 'core',
  'species': 'homo_sapiens',
  'length': 111,
  'start': 22003211},
 'start': 22003106,
 'object_type': 'Transcript',
 'Parent': 'ENSG00000211798',
 'biotype': 'TR_V_gene',
 'species': 'homo_sapiens',
 'is_canon

In [128]:
missings.sort_values(by='HEK293_TPM', ascending=False)[:20]

,AC,Entry,Gene,ENST,HEK293_TPM,ENSP
491,P0DMP1,HMN12_HUMAN,MTRNR2L12,ENST00000600213,3188.445000,ENSP00000468991
470,P0CJ75,HMN8_HUMAN,MTRNR2L8,ENST00000536684,547.973500,ENSP00000439666
463,P0CJ68,HMN1_HUMAN,MTRNR2L1,ENST00000540040,393.955500,ENSP00000439228
187,A4D0T7,SIM30_HUMAN,SMIM30,ENST00000397764,93.494150,ENSP00000490095
591,P60606,CTXN1_HUMAN,CTXN1 CTXN,ENST00000318978,60.018250,ENSP00000313226
1539,S4R3Y5,HMN11_HUMAN,MTRNR2L11,ENST00000604646,36.966950,ENSP00000474861
471,P0CJ77,HMN10_HUMAN,MTRNR2L10,ENST00000545075,32.505350,ENSP00000442159
1242,Q96DE9,EOLA2_HUMAN,EOLA2 CXorf40B,ENST00000370406,18.421850,ENSP00000359434
116,A0A1B0GUW7,SIM27_HUMAN,SMIM27 C9orf133 TOPORS-AS1,ENST00000450093,14.321300,ENSP00000490727
234,A6NHG4,DDTL_HUMAN,DDTL,ENST00000215770,13.280350,ENSP00000215770


In [129]:
missings[missings['AC']=='Q96DE9']

,AC,Entry,Gene,ENST,HEK293_TPM,ENSP
1240,Q96DE9,EOLA2_HUMAN,EOLA2 CXorf40B,ENST00000355203,9.03710,ENSP00000347339
1241,Q96DE9,EOLA2_HUMAN,EOLA2 CXorf40B,ENST00000370404,0.70219,ENSP00000359432
1242,Q96DE9,EOLA2_HUMAN,EOLA2 CXorf40B,ENST00000370406,18.42185,ENSP00000359434


In [130]:
'ENST00000462691' in hek['enstid'].to_list()

True